![pipeline](pictures/pictures.002.png)

# Text-Fabric from ETCBC

This notebook assembles the data from the ETCBC that is needed
to compile its datasets in text-fabric-format on GitHub.
Ultimately the data for the website [SHEBANQ](https://shebanq.ancient-data.org) will be
derived from these Text-Fabric-sources.

## Pipeline
This is **pipe 1** of the pipeline from ETCBC data to the website SHEBANQ.

A run of this pipe produces a data *version*.
It should be run whenever there are new or updated data sources present that affect the output data.
Since all input data is delivered in a GitHub repo, we have excellent machinery to 
work with versioning.

The pipe works by executing a series of programs, contained in GitHub repositories.
For each repository in the pipe, a series of notebooks will be executed.
See [script mode](https://github.com/ETCBC/pipeline/blob/master/README.md#operation) for 
details on how we call notebooks.

All this is specified in the configuration below.

### Core data

The core data is delivered by the ETCBC as `bhsa.mql.bz2` in 
the GitHub repo [bhsa](https://github.com/ETCBC/bhsa) in directory `source`.

This data will be converted by `tfFromMQL` in the `programs` directory.

The result of this action will be an updated Text-Fabric resource in its 
`tf/core` directory.

### Additional data

Researchers have contributed to the dataset, 
but not all that data is in the core.
They are typically in the repository where the research has been 
executed, and where the data is documented.

Before the pipe starts, these repos must be pulled.

## Continuous version
Version `c` acts as a *continuous* version. It will be overwritten
by new snapshots of the data on a regular basis.

We support the following workflow to carry out these updates:

1. make a new version called `_temp`. Note:
   * this choice of name prevents it to reach github, because `_temp` directories are in `.gitignore`;
   * after running this workflow, the version `_temp` already exists, this is not a problem;
2. put a new data snapshot in the `source/_temp` directory of the `bhsa` repo, add also data to the
   `source/_temp` directories of the other repos in the pipeline, as far as relevant;
3. run `good = runPipeline(pipeline, versions=['_temp'], force=True)`. Note:
   * we use `force=True` here, because then the old data in version `_temp` will be thoroughly overwritten;
4. if all went well run `copyVersion(pipeline, '_temp', 'c')`. This will overwrite all data directories
   in version `c` by the just created data directories in `temp`.

In [1]:
import os,sys,collections
from pipeline import runPipeline, copyVersion
from tf.fabric import Fabric

# Config

In [2]:
CORE_NAME = 'bhsa'

if 'SCRIPT' not in locals(): 
    SCRIPT = False
    DEFAULT_CORE_NAME = CORE_NAME
    DEFAULT_VERSION = 'c'

# Pipeline settings

Here all the nitty-gritty differences between versions are stated.

In [3]:
pipeline = dict(
    defaults = dict(
        CORE_NAME=CORE_NAME,
        VERSION=DEFAULT_VERSION,
        LANG_FEATURE='languageISO',
        OCC_FEATURE='g_cons',
        LEX_FEATURE='lex',
        TEXT_FEATURE='g_word_utf8',
        TRAILER_FEATURE='trailer_utf8',
        DO_VOCALIZED_LEXEME=True,
        EXTRA_OVERLAP='',
        LEX_FORMATS='@fmt:lex-trans-plain={lex0} ',
        RENAME=(
            ('g_suffix', 'trailer'),
            ('g_suffix_utf8', 'trailer_utf8'),
        ),
    ),
    versions={
        '_temp': dict(),
        'c': dict(),
        '2017': dict(),
        '2016': dict(),
        '4b': dict(
                LANG_FEATURE='language',
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
            ),
        '4': dict(
                LANG_FEATURE='language',
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
        ),
        '3': dict(
                LANG_FEATURE='language',
                OCC_FEATURE='surface_consonants',
                LEX_FEATURE='lexeme',
                TEXT_FEATURE='text',
                TRAILER_FEATURE='suffix',
                LEX_FORMAT='@fmt:lex-trans-plain={lexeme} ',
            ),
    },
    repoOrder = '''
        bhsa
        phono
        valence
        parallels
    ''',
    repoConfig = dict(
        bhsa=(
            dict(
                task='coreData',
            ),
            dict(
                task='bookNames',
                omit={},
            ),
            dict(
                task='lexicon',
                omit={'3'},
            ),
            dict(
                task='paragraphs',
                 omit={'3', '4', '4b'},
            ),
            dict(
                task='ketivQere',
                omit={'3', '4', '4b'},
            ),
            dict(
                task='stats',
                omit={'4', '4b'},
            ),
        ),
        phono=(
            dict(
                task='phono',
                omit={'3', '4', '4b'},
            ),
        ),
        valence=(
            dict(
                task='enrich',
                omit={'3'},
            ),
            dict(
                task='flowchart',
                omit={'3'},
            ),
        ),
        parallels=(
            dict(
                task='parallels',
                omit={},
                params=dict(
                    FORCE_MATRIX=False,
                ),
            ),
        ),
    ),
    repoDataDirs = dict(
        bhsa      = 'source _temp tf shebanq',
        phono     = '_temp tf',
        valence   = 'source _temp tf shebanq',
        parallels = 'source _temp tf',
    ),
)

# Run the pipeline

To run everything from scratch:

```python
good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)
```

To make a new version called `temp`

```python
good = runPipeline(pipeline, versions=['_temp'], force=True) # a new candidate for 'c'
```

To copy the `_temp` version over to the continuous version `c`:

```python
if good:
    copyVersion(pipeline, '_temp', 'c')
```
This will copy all data, `source`, `_temp` and `tf` over from `_temp` to `c`.

If you want to create the node mappings between versions,
go to the versionMappings notebook in the BHSA repo and run it.

In [4]:
# good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)
good = runPipeline(pipeline, versions=['2016', '2017', 'c'], force=True)


##############################################################################################
#                                                                                            #
#       0.00s Make version [2016]                                                            #
#                                                                                            #
##############################################################################################


**********************************************************************************************
*                                                                                            *
*       0.00s Make repo [bhsa]                                                               *
*                                                                                            *
**********************************************************************************************


---------------------------------------------

  0.23s 		otype half_verse
  0.23s 			feature label (str) =def=  : node
  0.24s 		otype verse
  0.24s 			feature verse (int) =def= 0 : node
  0.24s 			feature chapter (int) =def= 0 : node
  0.24s 			feature label (str) =def=  : node
  0.25s 			feature book (str) =def= Genesis : node
  0.25s 		otype phrase_atom
  0.25s 			feature number (int) =def= 0 : node
  0.25s 			feature dist (int) =def= 0 : node
  0.25s 			feature distributional_parent (str) =def= 0 : edge
  0.25s 			feature mother (str) =def= 0 : edge
  0.26s 			feature functional_parent (str) =def= 0 : edge
  0.26s 			feature det (str) =def= NA : node
  0.26s 			feature typ (str) =def= VP : node
  0.26s 			feature rela (str) =def= NA : node
  0.26s 			feature dist_unit (str) =def= clause_atoms : node
  0.26s 		objects in word
  6.68s 	line   1000000
    13s 	line   2000000
    16s 		objects in word
    18s 	line   3000000
    24s 	line   4000000
    30s 	line   5000000
    31s 		objects in word
    37s 	line   6000000
    43s 	l

   |     0.78s T prs_ps               to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.75s T ps                   to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.68s T qere                 to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.69s T qere_utf8            to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     1.32s T rela                 to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.76s T sp                   to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.75s T st                   to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.17s T tab                  to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.73s T trailer              to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.74s T trailer_utf8         to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     0.18s T txt                  to /Users/dirk/github/etcbc/bhsa/_temp/2016/tf
   |     1.35s T typ                  to /Users/dirk/github/etcbc

|      4m 41s pdp                       ... no changes
|      4m 42s pfm                       ... no changes
|      4m 42s prs                       ... no changes
|      4m 42s prs_gn                    ... no changes
|      4m 43s prs_nu                    ... no changes
|      4m 43s prs_ps                    ... no changes
|      4m 44s ps                        ... no changes
|      4m 44s qere                      ... differences after the metadata
|      4m 44s 	line      2 OLD -->3897	HAJ:Y;74><--
|      4m 44s 	line      2 NEW --><--
|      4m 44s 	line      3 OLD -->4420	>@H:@LO75W<--
|      4m 44s 	line      3 NEW --><--
|      4m 44s 	line      4 OLD -->5645	>@H:@LO92W<--
|      4m 44s 	line      4 NEW --><--
|      4m 44s 	line      5 OLD -->5912	>@95H:@LOW03<--
|      4m 44s 	line      5 NEW --><--

|      4m 44s qere_utf8                 ... differences after the metadata
|      4m 44s 	line      2 OLD -->3897	הַיְצֵ֣א<--
|      4m 44s 	line      2 NEW --><--
|      4m 

   |     1.43s T nu                   from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     2.42s T number               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.45s T pdp                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.41s T pfm                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.45s T prs                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.46s T prs_gn               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.46s T prs_nu               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.48s T prs_ps               from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.46s T ps                   from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.65s T qere                 from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.73s T qere_utf8            from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     2.52s T rela                 from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     1.47s T sp         

...

|     41m 33s 	Destination /Users/dirk/github/etcbc/valence/tf/c/.tf/valence.tfx exists
|     41m 33s NOTE: repo seems up to date. Will be run because of "force=True"
True True
..............................................................................................
.     41m 33s Load the existing TF dataset                                                   .
..............................................................................................
This is Text-Fabric 3.1.1
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

114 features found and 0 ignored
  0.00s loading features ...
   |     0.24s B lex_utf8             from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.18s B lex                  from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.01s B gloss                from /Users/dirk/github/etcbc/bhsa/tf/

|     42m 25s 	Done
|     42m 25s 	Phrases of kind C :  19302
|     42m 25s 	Phrases of kind L :  11680
|     42m 25s 	Phrases of kind I :   6018
|     42m 25s 	Total complements :  37000
|     42m 25s 	Total phrases     : 214541
..............................................................................................
.     42m 25s Checking enrichment logic                                                      .
..............................................................................................
|     42m 25s 	All 6 rules OK
..............................................................................................
.     42m 25s Generating enrichments                                                         .
..............................................................................................
|     42m 31s 	Generated enrichment values for 1380 verbs:
|     42m 31s 	Enriched values for 221366 nodes
|     42m 31s 	Overview of rule applications:
|     42m 31s gen

   |     0.21s B typ                  from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.07s B function             from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.78s T valence              from /Users/dirk/github/etcbc/valence/tf/c
   |     0.76s T predication          from /Users/dirk/github/etcbc/valence/tf/c
   |     0.78s T grammatical          from /Users/dirk/github/etcbc/valence/tf/c
   |     0.38s T original             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.51s T lexical              from /Users/dirk/github/etcbc/valence/tf/c
   |     0.52s T semantic             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.38s T f_correction         from /Users/dirk/github/etcbc/valence/tf/c
   |     0.38s T s_manual             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.48s T cfunction            from /Users/dirk/github/etcbc/valence/tf/c
   |     0.00s Feature overview: 119 for nodes; 4 for edges; 1 configs; 7 computed
    11s All features loaded/comp

|     43m 16s 	10000 clauses
|     43m 19s 	20000 clauses
|     43m 22s 	30000 clauses
|     43m 25s 	40000 clauses
|     43m 27s 	47385 clauses
..............................................................................................
.     43m 27s Writing sense feature to TF                                                    .
..............................................................................................
   |     0.10s T sense                to /Users/dirk/github/etcbc/valence/_temp/c/tf
..............................................................................................
.     43m 28s Check differences with previous version                                        .
..............................................................................................
|     43m 28s 	no features to add
|     43m 28s 	no features to delete
|     43m 28s 	1 features in common
|     43m 28s sense                     ... no changes
|     43m 28s Done
...................

|     43m 46s Generated 6215 notes
..............................................................................................
.     43m 46s Check differences with previous version                                        .
..............................................................................................
|     43m 46s 	no features to add
|     43m 46s 	no features to delete
|     43m 46s 	3 features in common
|     43m 46s crossref                  ... no changes
|     43m 46s crossrefLCS               ... no changes
|     43m 46s crossrefSET               ... no changes
|     43m 46s Done
..............................................................................................
.     43m 46s Deliver data set to /Users/dirk/github/etcbc/parallels/tf/c                    .
..............................................................................................
..............................................................................................
.     43m

# Consolidate the continuous version
If you have run an update version called `_temp`, and all has went well
you can copy over the entire version (including its source and temp directories to `c`).

In [ ]:
#good = True
#good = False

if good:
    copyVersion(pipeline, '_temp', 'c')